# Download Western Celebrity Face Images and Check for Image Quality

### In this file, we use a library called simple image download to download celebrity face images. 
### Then we make an s3 bucket to store these images.
### Then, we filter these images using Rekognition's detect faces method in order to cull low-quality images (i.e., images where multiple faces are detected, images that are too small, etc.)

In [2]:
# Let us install the library we are going to use to download images:
! pip install simple_image_download # Python library that simplifies the process of downloading images from the internet
! pip install scikit-image # collection of algorithms for image processing

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 25.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 kB 22.0 MB/s eta 0:00:00


In [2]:
import pandas as pd # library for data manipulation
import boto3 # Amazon Web Services (AWS) Software Development Kit (SDK) for Python; used to interact with AWS services, such as S3 (Simple Storage Service),
import json # imports the built-in json module, which provides methods for encoding and decoding JSON data
import s3fs # imports the s3fs library, which is a Python interface to interact with data stored on Amazon S3
import io # imports the built-in io module, which provides the Python interfaces for stream handling.
import os

import numpy as np
from IPython.display import Image as IImage

from PIL import Image, ImageDraw, ExifTags, ImageColor


In [4]:
print(f"Boto3 Version: {boto3.__version__}")
! pip install --upgrade boto3


Boto3 Version: 1.28.80
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 29.3 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.80
    Uninstalling botocore-1.31.80:
      Successfully uninstalled botocore-1.31.80
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.80
    Uninstalling boto3-1.28.80:
      Successfully uninstalled boto3-1.28.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.80 requires botocore==1.31.80, but you have botocore 1.32.6 which is incompatible.


In [5]:
# Please download these CSV files from the github 

# Link to the github: https://github.com/sophiaxxiao/Celebrity_recognition_project.git

# You can type in the terminal 'git clone https://github.com/sophiaxxiao/Celebrity_recognition_project.git' to obtain these files


# Make sure you have the correct paths 

western_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/western_celebs.csv')


In [6]:
western_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Diddy,$130 million,Musician,U.S.
1,2,Beyoncé,$105 million,Musician,U.S.
2,3,J.K. Rowling,$95 million,Author,U.K.
3,4,Drake,$94 million,Musician,Canada
4,6,The Weeknd,$92 million,Musician,Canada
...,...,...,...,...,...
75,95,Chance The Rapper,$33 million,Musician,U.S.
76,95,Katy Perry,$33 million,Musician,U.S.
77,97,Carmelo Anthony,$32.6 million,Athlete,U.S.
78,98,Jason Aldean,$32.5 million,Musician,U.S.


In [9]:
from simple_image_download import simple_image_download as simp 
response = simp.simple_image_download



In [12]:
# The code below downloads the images
for i in range(10, 30, 10):
    western_celebrity_names = western_celebs['Name'][i: i + 10] # selecting the top 14 celebrities 
    for name in western_celebrity_names:
        response().download(name , 30)


In [13]:
!ls simple_images

Adam Sandler	   Drake	    James Patterson  Mark Wahlberg
Adele		   Dr. Phil McGraw  Jerry Seinfeld   Paul McCartney
Andrew Luck	   Dwayne Johnson   Jimmy Buffett    Ryan Seacrest
Beyoncé		   Ellen DeGeneres  J.K. Rowling     The Weeknd
Bruce Springsteen  Elton John	    Justin Bieber    Vin Diesel
Chris Rock	   Garth Brooks     Kevin Durant
David Copperfield  Gordon Ramsay    LeBron James
Diddy		   Howard Stern     Louis C.K.


In [3]:
client=boto3.client('rekognition')
import subprocess
import re
import s3fs 


In [4]:
bucket_name = 'western-celebs' # (you need to change the name of the bucket -- make sure your bucket name is unique)
foldername = 'simple_images'
# Use this commmand to make the bucket 
!aws s3 mb s3://$bucket_name

# This is the AWS CLI command to create an S3 bucket. 
# The mb stands for "make bucket," and it's used to create a new S3 bucket.
!aws s3 mv ./$foldername s3://$bucket_name/ --recursive # move the contents of the folder into the s3 bucket

make_bucket: western-celebs

The user-provided path ./simple_images does not exist.


In [27]:
import boto3
from PIL import Image
from io import BytesIO

# Initialize the S3 client
s3 = boto3.client('s3')
fs = s3fs.S3FileSystem(anon=False)

bucket_name = 'western-celebs'

# List objects in the bucket
response = s3.list_objects_v2(Bucket= bucket_name)

# Initialize Rekognition client
rekognition_client = boto3.client('rekognition')

# Function to remove photo based on your criteria
def should_remove_photo(photo, bucket):
    # Perform label detection using Rekognition
    try:
        response = rekognition_client.detect_faces(Image={'S3Object': {'Bucket': bucket_name, 'Name': photo}})
    except rekognition_client.exceptions.InvalidImageFormatException as e:
        print(f"Invalid image format: {e}")
        return True
    # print(response)
    # Perform image checks
    if response['FaceDetails'] == []:
        return True
    
    image = Image.open(BytesIO(s3.get_object(Bucket=bucket_name, Key=photo)['Body'].read()))
    
    imgWidth, imgHeight = image.size

    face_details = response['FaceDetails'][0] if 'FaceDetails' in response and response['FaceDetails'] else None
    pitch = face_details['Pose']['Pitch'] if 'Pose' in face_details and 'Pitch' in face_details['Pose'] else None
    yaw = face_details['Pose']['Yaw'] if 'Pose' in face_details and 'Yaw' in face_details['Pose'] else None
    sunglasses = face_details.get('Sunglasses', {}).get('Value', None)
    bounding_box = face_details.get('BoundingBox', {})
    width = imgWidth * bounding_box.get('Width', 0)
    height = imgHeight * bounding_box.get('Height', 0)
    eyes_open = face_details.get('EyesOpen', {}).get('Value', None)
    number_faces_detected = len(response['FaceDetails'])
    occlusion_confidence = response['FaceDetails'][0].get('Occlusion', {}).get('Value', 0)

    
    # Check if Emotions array is present and non-empty
    emotions = face_details.get('Emotions', [])
    emotion = emotions[0]['Type'] if emotions else None
    emotion_confidence = emotions[0]['Confidence'] if emotions else None
    
    mouth_open = face_details.get('MouthOpen', {}).get('Value', None)
    
    if number_faces_detected > 1:
        print('Multiple faces detected')
        return True
    if width < 50 or height < 50:
        print('Too small')
        return True
    if abs(yaw) > 45:
        print('yaw too much')
        return True
    if pitch > 60 or pitch < -60:
        print('pitch too severe')
        return True
    if occlusion_confidence > 70:
        print('face occluded')
        return True
    return False


In [52]:
response = s3.list_objects_v2(Bucket= bucket_name)

len(response.get('Contents', []))

644

In [53]:
# Iterate through the objects in the bucket
for obj in response.get('Contents', [])[500:]:
    # Get the object key (file path)
    object_key = obj['Key']
    print(object_key)

    # Apply your function to the object
    if should_remove_photo(object_key, bucket_name):
        # Remove the file from the bucket
        s3.delete_object(Bucket=bucket_name, Key=object_key)
        
        print("Photo removed.")
    else:
        print("Keeping photo:", object_key)


Louis C.K./Louis C.K._26.jpg
Keeping photo: Louis C.K./Louis C.K._26.jpg
Louis C.K./Louis C.K._27.jpg
Keeping photo: Louis C.K./Louis C.K._27.jpg
Louis C.K./Louis C.K._28.jpg
Keeping photo: Louis C.K./Louis C.K._28.jpg
Louis C.K./Louis C.K._5.jpg
Keeping photo: Louis C.K./Louis C.K._5.jpg
Louis C.K./Louis C.K._6.jpg
Keeping photo: Louis C.K./Louis C.K._6.jpg
Louis C.K./Louis C.K._7.jpg
Keeping photo: Louis C.K./Louis C.K._7.jpg
Louis C.K./Louis C.K._8.jpg
Keeping photo: Louis C.K./Louis C.K._8.jpg
Louis C.K./Louis C.K._9.jpg
Keeping photo: Louis C.K./Louis C.K._9.jpg
Mark Wahlberg/Mark Wahlberg_13.jpg
Keeping photo: Mark Wahlberg/Mark Wahlberg_13.jpg
Mark Wahlberg/Mark Wahlberg_14.jpg
Keeping photo: Mark Wahlberg/Mark Wahlberg_14.jpg
Mark Wahlberg/Mark Wahlberg_16.jpg
Keeping photo: Mark Wahlberg/Mark Wahlberg_16.jpg
Mark Wahlberg/Mark Wahlberg_18.jpg
Keeping photo: Mark Wahlberg/Mark Wahlberg_18.jpg
Mark Wahlberg/Mark Wahlberg_19.jpg
Keeping photo: Mark Wahlberg/Mark Wahlberg_19.jpg
M

In [5]:
def delete_excess_objects(bucket_name, max_items_per_folder=10):
    s3 = boto3.client('s3')

    # List objects at the root level
    objects = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

    # Iterate through each "folder" (common prefix)
    for common_prefix in objects.get('CommonPrefixes', []):
        folder_key = common_prefix['Prefix']
        print(f"Processing folder: {folder_key}")

        # List all objects in the folder
        folder_objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)

        # Sort objects by last modified time
        sorted_objects = sorted(folder_objects.get('Contents', []), key=lambda x: x['LastModified'])

        # Keep the last max_items_per_folder objects
        objects_to_keep = sorted_objects[-max_items_per_folder:]

        # Delete excess items in the folder
        for obj in sorted_objects[:-max_items_per_folder]:
            obj_key = obj['Key']
            print(f"Deleting excess item: {obj_key}")
            s3.delete_object(Bucket=bucket_name, Key=obj_key)


In [7]:
bucket_name = 'western-celebs'

delete_excess_objects(bucket_name, max_items_per_folder=5)


Processing folder: Adam Sandler/
Deleting excess item: Adam Sandler/Adam Sandler_17.jpg
Deleting excess item: Adam Sandler/Adam Sandler_20.jpg
Deleting excess item: Adam Sandler/Adam Sandler_21.jpg
Deleting excess item: Adam Sandler/Adam Sandler_22.jpg
Deleting excess item: Adam Sandler/Adam Sandler_29.jpg
Processing folder: Adele/
Deleting excess item: Adele/Adele_20.jpg
Deleting excess item: Adele/Adele_21.jpg
Deleting excess item: Adele/Adele_26.jpg
Deleting excess item: Adele/Adele_27.jpg
Deleting excess item: Adele/Adele_28.jpg
Processing folder: Andrew Luck/
Deleting excess item: Andrew Luck/Andrew Luck_15.jpg
Deleting excess item: Andrew Luck/Andrew Luck_16.jpg
Deleting excess item: Andrew Luck/Andrew Luck_18.jpg
Deleting excess item: Andrew Luck/Andrew Luck_19.jpg
Deleting excess item: Andrew Luck/Andrew Luck_23.jpg
Processing folder: Beyoncé/
Deleting excess item: Beyoncé/Beyoncé_41.jpg
Deleting excess item: Beyoncé/Beyoncé_43.jpg
Deleting excess item: Beyoncé/Beyoncé_44.jpg
D